Importing libraries

In [5]:
import json as js
from collections import defaultdict
import string
from keras import layers #type : ignore

loading paths

In [6]:
with open('Config_RNN.json','r') as file:
    paths = js.load(file)

Loading initial train data

In [3]:
with open(paths["Caption_Train"],'r') as file:
    train_data = js.load(file)

Extracting imp data

In [4]:
newdict = {}
newdict["img_data"] = train_data["images"]
newdict["annotations_data"] = train_data["annotations"]

Creating Corpus

In [5]:
img_data = newdict["img_data"]
annotations_data = newdict["annotations_data"]
captions_dict = defaultdict(list)

for ann in annotations_data:
    captions_dict[ann["image_id"]].append(ann["caption"])

image_caption_data = []
for img in img_data:
    img_id = img["id"]
    image_info = {
        "id": img_id,
        "file_name": img["file_name"],
        "captions": captions_dict[img_id]  
    }
    image_caption_data.append(image_info)

with open(paths["Corpus"], "w") as f:
    js.dump(image_caption_data, f, indent=4)


Text Vectorization

In [9]:
with open(paths["Corpus"],'r') as file:
    data = js.load(file)

def clean_caption(caption):
    caption = caption.lower()
    caption = caption.translate(str.maketrans('', '', string.punctuation))
    caption = caption.strip()
    return caption

caption_dict = {}

for item in data:
    captions = [clean_caption(c) for c in item['captions']]
    caption_dict[item['file_name']] = captions

all_captions = []

for captions in caption_dict.values():
    for caption in captions:
        all_captions.append(f'startseq {caption} endseq')

vectorizer = layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=30 
)

vectorizer.adapt(all_captions)
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)

with open(paths["Preprocessed_Corpus"], "w") as f:
    js.dump(vocab, f)

print("Vocabulary Size:", vocab_size)

Vocabulary Size: 10000
